# Evolver Loop 11 Analysis

Current state:
- Best CV: 70.630478
- Target: 68.919154
- Gap: 1.711 points (2.42%)

All public sources exhausted. Need to find NEW approaches.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

print("Libraries loaded")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

print("ChristmasTree class defined")

In [ ]:
def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('s')
        y = str(row["y"]).lstrip('s')
        deg = str(row["deg"]).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def calculate_total_score(df):
    total = 0
    for n in range(1, 201):
        trees = load_trees(n, df)
        total += calculate_score(trees)
    return total

print("Helper functions defined")

In [ ]:
# Scan ALL snapshots for valid submissions
snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshots = [d for d in os.listdir(snapshot_dir) if os.path.isdir(os.path.join(snapshot_dir, d))]

print(f"Found {len(snapshots)} snapshots")

# Check each snapshot for submission.csv
results = []
for snap in snapshots:
    sub_path = os.path.join(snapshot_dir, snap, 'submission', 'submission.csv')
    if os.path.exists(sub_path):
        try:
            df = pd.read_csv(sub_path)
            if 'id' in df.columns and 'x' in df.columns:
                # Quick score calculation (just check first few N values)
                score = calculate_total_score(df)
                results.append((snap, sub_path, score))
                print(f"{snap}: score = {score:.6f}")
        except Exception as e:
            print(f"{snap}: ERROR - {e}")

print(f"\nFound {len(results)} valid submissions")

In [ ]:
# Sort by score and show top 10
results.sort(key=lambda x: x[2])
print("Top 10 snapshots by score:")
for snap, path, score in results[:10]:
    print(f"  {snap}: {score:.6f}")

In [ ]:
# Check the best snapshot for overlaps
if results:
    best_snap, best_path, best_score = results[0]
    print(f"\nBest snapshot: {best_snap} with score {best_score:.6f}")
    
    df = pd.read_csv(best_path)
    
    # Check for overlaps
    overlap_count = 0
    for n in range(1, 201):
        trees = load_trees(n, df)
        if has_collision(trees):
            overlap_count += 1
    
    print(f"Overlapping N values: {overlap_count}")
    
    if overlap_count == 0:
        print("\n*** VALID SUBMISSION FOUND! ***")
        print(f"Score: {best_score:.6f}")
        print(f"Path: {best_path}")
    else:
        print(f"\nInvalid - has {overlap_count} overlapping N values")

In [ ]:
# Compare with current best
current_best = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')
current_score = calculate_total_score(current_best)
print(f"Current best score: {current_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {current_score - 68.919154:.6f}")